# Лабораторная работа №1

## Задания

Самостоятельно написать код, реализующий искусственный нейрон с сигма-функцией активации, и возможность строить на его основе многослойные сети. Код должен также реализовывать градиентный спуск и обратное распространение ошибки.

На основе вашего кода:

1. Решить задачу  классификации датаcета Iris одним нейроном.
2. Решить задачу  классификации датаcета Iris одним  нейросетью из 2 слоев по 10 нейронов в слое.
3. Отрисовать разделяющую линию для обеих моделей. Сравнить метрики классификации.

## Реализация

### Библиотеки

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

### Сигма-функция

Используется в качестве функции активации

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [3]:
def der_sigmoid(x):  # производная сигма-функции
    return sigmoid(x) * (1 - sigmoid(x))

### Функция ошибок

Здесь будет использоваться MSE (Mean Square Error)

In [4]:
def mse(y_true, y_pred):
    return ((y_true - y_pred)**2).mean()

In [5]:
def der_mse(y_true, y_pred):  # производная функции ошибок
    return -2 * (y_true - y_pred).mean()

### Слой нейронной сети

Хранит матрицу весов нейронов, матрицу сдвигов нейронов, матрицу ошибок и вывод нейронов

In [6]:
class Layer:
    def __init__(self, neuron_count):
        self.neuron_count = neuron_count
        self.weights = None  # матрица весов нейронов N x W, где N - кол-во нейронов в слое, а W - кол-во входных данных в слой
        self.biases = None  # вектор сдвигов нейронов N x 1, где N - кол-во нейронов в слое
        self.output = None  # вектор вывода N x 1
        self.error = None  # вектор ошибок N x 1, нужен для обратного распространения ошибки в градиентном спуске

    
    def setup_layer(self, input_count):
        self.weights = np.random.rand(self.neuron_count, input_count)  # веса по умолчанию
        self.biases = np.random.rand(self.neuron_count, 1)  # сдвиг по умолчанию
        self.weights = np.full((self.neuron_count, input_count), 1, dtype=np.float64)
        self.biases = np.zeros((self.neuron_count, 1), dtype=np.float64)

    
    def input_output(self, input_values):
        self.output = sigmoid((self.weights @ input_values) + self.biases)  # вектор вывода, здесь F(WX + B), где X - вектор входных данных, W - матрица весов, B - вектор сдвигов
        
        return self.output

### Нейронная сеть

Сеть состоит из слоёв, здесь определяется её поведение

In [7]:
class NeuralNetwork:
    def __init__(self, layers=[]):
        self.layers = layers

    
    def back_propogation(self, true_values):  # обратное распространение ошибки
        for layer in range(len(self.layers) - 1, -1, -1):  # идём в обратном порядке, в этом суть метода
            cur_layer = self.layers[layer]
            
            if layer == len(self.layers) - 1:
                # Для выходного слоя вектор ошибок будет выглядеть так: L'(t, o) * f'(o), где:
                # L' - производная функции ошибок, f' - производная функции активации
                # o - вывод значений нынешнего слоя, t - истинный вывод
                
                cur_layer.error = der_mse(true_values, cur_layer.output) * der_sigmoid(cur_layer.output)

            else:
                # Для остальных слоёв вектор ошибок будет такой: TD * f'(o), где:
                # D - вектор ошибок следующего слоя, T - транспонированная матрица весов следующего слоя
                # f' - производная функции активации, o - вывод значений нынешнего слоя
                
                next_layer = self.layers[layer + 1]
                cur_layer.error = (next_layer.weights.transpose() @ next_layer.error) * der_sigmoid(cur_layer.output)
                
    
    def gradient_descent(self, input_values, learning_rate):  # градиентный спуск
        for layer in range(len(self.layers)):  # теперь идём по порядку, так называемый forward progogation     
            cur_layer = self.layers[layer]

            # Изменение сдвигов равно E*l, где l - скорость обучения, E - вектор ошибок

            delta_biases = cur_layer.error * learning_rate 

            # Изменение весов равно ET*l, где l - скорость обучения, E - вектор ошибок, T - транспонированная матрица вводных данных слоя

            delta_weights = None

            if layer == 0:
                delta_weights = (cur_layer.error @ input_values.transpose()) * learning_rate

            else:
                prev_layer = self.layers[layer - 1]
                delta_weights = (cur_layer.error @ prev_layer.output.transpose()) * learning_rate

            cur_layer.weights -= delta_weights
            cur_layer.biases -= delta_biases

        
    def fit(self, X_train, y_train, learning_rate=0.1, epochs=1000, verbose=False, early_stopping=None):
        for layer in range(len(self.layers)):  # инициализация слоёв
            if layer == 0:
                self.layers[layer].setup_layer(len(X_train[0]))  # первый слой - входной, у него кол-во входных данных равно кол-ву вводимых атрибутов

            else:
                self.layers[layer].setup_layer(self.layers[layer-1].neuron_count)  # у каждого последующего слоя кол-во входных данных равно кол-ву нейронов на предыдущем слое
        
        last_error = None

        for epoch in range(epochs):  # начало обучения
            y_pred = []
            
            for value in range(len(X_train)):
                input_values = np.array([X_train[value]]).transpose()  # входные данные в виде вектор-столбца
                true_values = np.array([y_train[value]]).transpose()  # истинные данные в виде вектор-столбца
                
                neuron_output = input_values  # в первый слой идут входные данные

                for layer in range(len(self.layers)):       
                    neuron_output = self.layers[layer].input_output(neuron_output)

                y_pred.append(neuron_output.transpose()[0])  # значение последнего слоя и будет предсказанным значением
                
                self.back_propogation(true_values)  # считаем обратное распространение ошибки
                self.gradient_descent(input_values, learning_rate)  # выполняем градиентный спуск

            y_pred = np.array(y_pred)
            error = mse(y_train, y_pred)

            if verbose:
                print(f"Epoch {epoch+1} out of {epochs}, current error is {error}")

            if early_stopping is not None:  # early stopping - минимальное значение разницы ошибок последних двух итераций, меньше которого обучение останавливается
                if last_error is not None:
                    if abs(error - last_error) < early_stopping:
                        if verbose:
                            print("Stopping early")
                        return

                last_error = error

    
    def predict(self, X):
        y_pred = []  # предсказанные значения

        for value in range(len(X)):            
            neuron_output = np.array([X_train[value]]).transpose()  # входные данные в виде вектор-столбца

            for layer in range(len(self.layers)):    
                neuron_output = self.layers[layer].input_output(neuron_output)

            y_pred.append(neuron_output.transpose()[0])  # значение последнего слоя и будет предсказанным значением

        return np.array(y_pred)  # получили предсказание

### Датасет

In [8]:
df = load_iris()
X, y = df["data"], df["target"]
types = len(df["target_names"])  # т.к. решаем задачу классификации, то нужно знать, сколько всего возможных типов классификации вообще есть

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

### Преобразование датасета

X и y нужно преобразовать перед тем, как работать с ними

Предположим, для задачи классификации есть 4 возможных вывода (0, 1, 2, 3) и y выглядит так:

```
| y |
|---|
| 0 |
| 2 |
| 1 |
| 2 | 
| 3 |
| 1 |
```

Нейроны выдают значения от 0 до 1, и чтобы классификация прошла правильно, преобразуем данные в матрицу вхождений

```
| y |     |    y    |
|---|     |---------|
| 0 | --> | 1 0 0 0 |
| 2 | --> | 0 0 1 0 |
| 1 | --> | 0 1 0 0 |
| 2 | --> | 0 0 1 0 |
| 3 | --> | 0 0 0 1 |
| 1 | --> | 0 1 0 0 | 
```

In [10]:
def hot_transform(y, types):
    N = len(y)
    
    transformed = np.zeros((N, types))

    for i in range(N):
        transformed[i][y[i]] = 1

    return transformed

In [11]:
y_train = hot_transform(y_train, types)

In [12]:
y_test = hot_transform(y_test, types)
y_test

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]])

### Сеть из 1 слоя с 1 нейроном

In [13]:
model1 = NeuralNetwork([
    Layer(1),
    Layer(types)  # кол-во нейронов на выходном слою равно кол-ву типов классификации
])

In [14]:
model1.fit(X_train, y_train, verbose=True, learning_rate=0.01, epochs=2000, early_stopping=10**-7)

Epoch 1 out of 2000, current error is 0.35026848302839964
Epoch 2 out of 2000, current error is 0.2997740689424805
Epoch 3 out of 2000, current error is 0.2624126621784348
Epoch 4 out of 2000, current error is 0.23813879929280996
Epoch 5 out of 2000, current error is 0.22773988391230288
Epoch 6 out of 2000, current error is 0.22451459424128803
Epoch 7 out of 2000, current error is 0.22339825576733338
Epoch 8 out of 2000, current error is 0.22290804293800043
Epoch 9 out of 2000, current error is 0.22265314935223712
Epoch 10 out of 2000, current error is 0.22250574687447025
Epoch 11 out of 2000, current error is 0.2224144929425774
Epoch 12 out of 2000, current error is 0.22235537384839535
Epoch 13 out of 2000, current error is 0.2223158496955622
Epoch 14 out of 2000, current error is 0.22228882415872586
Epoch 15 out of 2000, current error is 0.22227003582293572
Epoch 16 out of 2000, current error is 0.22225680946845303
Epoch 17 out of 2000, current error is 0.22224740820588468
Epoch 18 o

In [15]:
y_pred1 = model1.predict(X_test)
y_pred1

array([[0.33302718, 0.33302718, 0.33302718],
       [0.33360636, 0.33360636, 0.33360636],
       [0.3336356 , 0.3336356 , 0.3336356 ],
       [0.33365999, 0.33365999, 0.33365999],
       [0.3336388 , 0.3336388 , 0.3336388 ],
       [0.33518739, 0.33518739, 0.33518739],
       [0.33381672, 0.33381672, 0.33381672],
       [0.33394192, 0.33394192, 0.33394192],
       [0.33491982, 0.33491982, 0.33491982],
       [0.3343213 , 0.3343213 , 0.3343213 ],
       [0.3335038 , 0.3335038 , 0.3335038 ],
       [0.33253642, 0.33253642, 0.33253642],
       [0.33378293, 0.33378293, 0.33378293],
       [0.33436936, 0.33436936, 0.33436936],
       [0.33257502, 0.33257502, 0.33257502],
       [0.33395714, 0.33395714, 0.33395714],
       [0.33319749, 0.33319749, 0.33319749],
       [0.33386943, 0.33386943, 0.33386943],
       [0.33547449, 0.33547449, 0.33547449],
       [0.33319704, 0.33319704, 0.33319704],
       [0.33257246, 0.33257246, 0.33257246],
       [0.33321445, 0.33321445, 0.33321445],
       [0.

### Сеть из 2 слоёв по 10 нейронов

In [16]:
model2 = NeuralNetwork([
    Layer(10),
    Layer(10),
    Layer(types)  # кол-во нейронов на выходном слою равно кол-ву типов классификации
])

In [17]:
model2.fit(X_train, y_train, verbose=True, learning_rate=0.01, epochs=2000, early_stopping=10**-7)

Epoch 1 out of 2000, current error is 0.6611339086395339
Epoch 2 out of 2000, current error is 0.5610690637807318
Epoch 3 out of 2000, current error is 0.45371215203105153
Epoch 4 out of 2000, current error is 0.35228268865801343
Epoch 5 out of 2000, current error is 0.2870329413791231
Epoch 6 out of 2000, current error is 0.25329990513893874
Epoch 7 out of 2000, current error is 0.23719122343138915
Epoch 8 out of 2000, current error is 0.22956422976377083
Epoch 9 out of 2000, current error is 0.22589317899639866
Epoch 10 out of 2000, current error is 0.2240879607228091
Epoch 11 out of 2000, current error is 0.22318276949921667
Epoch 12 out of 2000, current error is 0.22272161564604448
Epoch 13 out of 2000, current error is 0.22248376858374014
Epoch 14 out of 2000, current error is 0.2223599469073991
Epoch 15 out of 2000, current error is 0.2222950358187954
Epoch 16 out of 2000, current error is 0.22226083137216568
Epoch 17 out of 2000, current error is 0.2222427387995529
Epoch 18 out 

In [18]:
y_pred2 = model2.predict(X_test)
y_pred2

array([[0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.33356059, 0.33356059, 0.33356059],
       [0.